# Interpreting the input dataset

In the given dataset we observed that some cells had no value. For the sales file, we saw that for each pair of SKU and geoCluster there was usually at least a day with a
value for the sale and price cell. We assumed the client sold a certain product that specific day and had the intention of selling it for the next days (where no price and sales are 
registered).
We have not deleted those entries because we considered that they provided important information.\
On how we filled those null values, we though a good aproximation was the mean of the values that were actually in the table for that combination of SKU and geoClient.

# Importing dependencies

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt


tf.get_logger().setLevel('ERROR')

# Reading input datasets

In [3]:
df_geo_params = pd.read_csv('geo_params.csv')
df_sales = pd.read_csv('sales.csv')
df_sku = pd.read_csv('sku.csv')
df_test = pd.read_csv('test.csv')

# Cleaning datasets

## Sales dataset

### Removed unnecessary columns
- ID

### Null values treatment
For each **SKU** and **geoCluster**, rows that have null values on **price** and **sales** have assigned the mean values of the rows that contain those values.

## SKU dataset

### Removed unnecessary columns
- lagerUnitQuantity, Units, Type, trademark, brandId

### Null values treatment
- **Category**: as the **Group** of all the null values is **Youghurt**, they are assigned to the **Youghurts** category.
- **countryOfOrigin**: null values have a new symbolic value of **0.0** assigned.

In [4]:
df_sales = df_sales.drop(labels='ID', axis=1)
df_grouped = df_sales.groupby(['SKU', 'geoCluster'])
df_means = df_grouped.mean()
for index, row in df_means.iterrows():
    df_filter = df_sales[(df_sales['SKU'] == index[0]) & (df_sales['geoCluster'] == index[1])]
    df_sales.loc[(df_sales['SKU'] == index[0]) & (df_sales['geoCluster'] == index[1]), 'price'] = df_filter['price'].fillna(row['price'])
    df_sales.loc[(df_sales['SKU'] == index[0]) & (df_sales['geoCluster'] == index[1]), 'sales'] = df_filter['sales'].fillna(row['sales'])

df_sku = df_sku.drop(labels=['lagerUnitQuantity', 'Units', 'Type', 'trademark', 'brandId'], axis=1)
df_sku['Category'] = df_sku['Category'].fillna('Yoghurts')
df_sku['countryOfOrigin'] = df_sku['countryOfOrigin'].fillna(0.0)

# Merging datasets

In [5]:
df_full = pd.merge(df_sales, df_geo_params, on='geoCluster', how='inner')
df_full = pd.merge(df_full, df_sku, on='SKU', how='inner')

# Removing few remaining NaN values

In [6]:
df_full = df_full.dropna()

# Changing date's strings to days since 1st Jan 2020

In [7]:
def getDays(date_string): return (datetime.strptime(date_string, '%Y-%m-%d') - datetime(2020, 1, 1)).days
df_full.loc[:, 'date'] = df_full['date'].apply(getDays)
df_full.loc[:, 'countryOfOrigin'] = df_full['countryOfOrigin'].astype('int')

# Normalizing the numerical features to their Z-scores

In [9]:
mean_df_full = df_full.loc[:, ['date', 'price', 'sales']].mean()
std_df_full = df_full.loc[:, ['date', 'price', 'sales']].std()
df_full.loc[:, ['date', 'price', 'sales']] = (df_full.loc[:, ['date', 'price', 'sales']] - mean_df_full) / std_df_full

# Creating training set and test set

In [11]:
df_training = df_full.sample(frac=0.8)
df_testing = df_full.drop(df_training.index)

# Creating feature layer

In [ ]:
feature_columns = []

fc_geoCluster = tf.feature_column.categorical_column_with_vocabulary_list("geoCluster", df_training['geoCluster'].unique())
fc_geoCluster = tf.feature_column.indicator_column(fc_geoCluster)
feature_columns.append(fc_geoCluster)

fc_SKU = tf.feature_column.categorical_column_with_vocabulary_list("SKU", df_training['SKU'].unique())
fc_SKU = tf.feature_column.indicator_column(fc_SKU)
feature_columns.append(fc_SKU)

resolution_in_Zs = 0.05

fc_date_as_numeric_column = tf.feature_column.numeric_column("date")
fc_date_boundaries = list(np.arange(int(min(df_training['date'])), 
                                     int(max(df_training['date'])), 
                                     resolution_in_Zs))
fc_date = tf.feature_column.bucketized_column(fc_date_as_numeric_column, fc_date_boundaries)

fc_price = tf.feature_column.numeric_column("price")
feature_columns.append(fc_price)

fc_cityId = tf.feature_column.categorical_column_with_vocabulary_list("cityId", df_training['cityId'].unique())
fc_cityId = tf.feature_column.indicator_column(fc_cityId)
feature_columns.append(fc_cityId)

fc_category = tf.feature_column.categorical_column_with_vocabulary_list("Category", df_training['Category'].unique())
fc_category = tf.feature_column.indicator_column(fc_category)
feature_columns.append(fc_category)

fc_countryOfOrigin = tf.feature_column.categorical_column_with_vocabulary_list("countryOfOrigin", df_training['countryOfOrigin'].unique())
fc_countryOfOrigin = tf.feature_column.indicator_column(fc_countryOfOrigin)
feature_columns.append(fc_countryOfOrigin)

fc_group = tf.feature_column.categorical_column_with_vocabulary_list("Group", df_training['Group'].unique())
fc_group = tf.feature_column.indicator_column(fc_group)
feature_columns.append(fc_group)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

# Callbacks


In [ ]:
patience_early_stopping = 5
patience_RLRL_callback = 2

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath = './models',
  monitor="loss",
  verbose=1,
  save_best_only=True,
  mode="auto",
  save_freq="epoch",
  options=None,
  )

early_stoping_callback = tf.keras.callbacks.EarlyStopping(  
  monitor="loss",
  patience=patience_early_stopping,
  verbose=1,
  mode="auto",
)

RLRO_callback = tf.keras.callbacks.ReduceLROnPlateau(
  monitor="loss",
  factor=0.025,
  patience=patience_RLRL_callback,
  verbose=1,
  mode="auto",
  min_delta=0.0001,
  cooldown=0,
  min_lr=0,
)

model_callbacks = [model_checkpoint_callback, early_stoping_callback, RLRO_callback]

# Building model functions

In [ ]:
def create_model(my_learning_rate, my_feature_layer):
    """Create and compile a simple linear regression model."""
    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the layer containing the feature columns to the model.
    model.add(my_feature_layer)

    # Describe the topography of the model by calling the tf.keras.layers.Dense
    # method once for each layer. We've specified the following arguments:
    #   * units specifies the number of nodes in this layer.
    #   * activation specifies the activation function (Rectified Linear Unit).
    #   * name is just a string that can be useful when debugging.

    # Define the first hidden layer with 20 nodes.   
    model.add(tf.keras.layers.Dense(units=20, activation='relu',name='Hidden1'))
    
    # Define the second hidden layer with 12 nodes. 
    model.add(tf.keras.layers.Dense(units=12, activation='relu', name='Hidden2'))

    # Define the output layer.
    model.add(tf.keras.layers.Dense(units=1, name='Output'))                              

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                  loss="mean_squared_error",
                  metrics=[tf.keras.metrics.MeanSquaredError()])

    return model         


def train_model(model, dataset, epochs, label_name, batch_size=None):
    """Train the model by feeding it data."""
    # Split the dataset into features and label.
    features = {name:np.array(value) for name, value in dataset.items()}
    label = np.array(features.pop(label_name))
    
    
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, callbacks=model_callbacks) 

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # To track the progression of training, gather a snapshot
    # of the model's mean squared error at each epoch. 
    hist = pd.DataFrame(history.history)
    mse = hist["mean_squared_error"]

    return epochs, mse

def plot_the_loss_curve(epochs, mse):
    """Plot a curve of loss vs. epoch."""

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Mean Squared Error")

    plt.plot(epochs, mse, label="Loss")
    plt.legend()
    plt.ylim([mse.min()*0.95, mse.max() * 1.03])
    plt.show()  

# Model training

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.1
epochs = 100
batch_size = 15000
label_name = "sales"

# Establish the model's topography.
my_model = create_model(learning_rate, feature_layer)

# Train the model on the normalized training set.
epochs, mse = train_model(my_model, df_training, epochs, label_name, batch_size)
plot_the_loss_curve(epochs, mse)

test_features = {name:np.array(value) for name, value in df_testing.items()}
test_label = np.array(test_features.pop(label_name)) # isolate the label
print("\n Evaluate the linear regression model against the test set:")
my_model.evaluate(x = test_features, y = test_label, batch_size=batch_size)

# Predicting test sales

In [57]:
df_test = pd.read_csv('test.csv')
df_test = pd.merge(df_test, df_geo_params, on='geoCluster', how='inner')
df_test = pd.merge(df_test, df_sku, on='SKU', how='inner')
df_test = df_test.drop(labels=['ID', 'sales'], axis=1)

def getDays(date_string): return (datetime.strptime(date_string, '%Y-%m-%d') - datetime(2020, 1, 1)).days
df_test.loc[:, 'date'] = df_test['date'].apply(getDays)
df_test.loc[:, 'countryOfOrigin'] = df_test['countryOfOrigin'].astype('int')
df_test.loc[:, 'countryOfOrigin'] = df_test['countryOfOrigin'].fillna(0.0)
df_test.loc[:, 'Category'] = df_test['Category'].fillna('Yoghurts')
df_test = df_test.rename(columns={'price_filled': 'price'})
df_test.loc[:, ['date', 'price']] = (df_test.loc[:, ['date', 'price']] - mean_df_full) / std_df_full

df_test.head()

,geoCluster,SKU,date,price,cityId,Category,countryOfOrigin,Group
0,21,32485,1.562082,-0.409281,1,Banana,0,Tropical fruits
1,21,32485,1.570105,-0.409281,1,Banana,0,Tropical fruits
2,21,32485,1.578127,-0.409281,1,Banana,0,Tropical fruits
3,21,32485,1.586150,-0.409281,1,Banana,0,Tropical fruits
4,21,32485,1.594173,-0.409281,1,Banana,0,Tropical fruits


In [58]:
test_features = {name:np.array(value) for name, value in df_test.items()}
test_results = my_model.predict(test_features, verbose=1)
test_results = test_results * std_df_full['sales'] + mean_df_full['sales']

df_test_results = pd.read_csv('test.csv')
df_test_results['sales'] = test_results
df_test_results.to_csv('test.csv', index=False)

6336/6336 [==============================] - 51s 8ms/step


# Interpreting the model results

When we got the predictions for the sales, we were able to extract conclusions and ideas on how to improve them and the general benefit for our client.
Some of our ideas are:


*   If the sales' expectancy for a product is high, we see fit to make a market analysis, using our own predictor, on how the rise or reduction of the prize would affect the sales made. In any case, a reduction on the advertisement would be recommended as the product may not need it.

*    On the contrary, if the sales' expectancy for a product is low, another market analysis would be advised but decreasing the price and analyze if by doing so, the sales would rise and the client wouldn't loose money by doing so. 

*    In any case, following the previous point, we see necessary an advertisement boost for that product. That way, we could increment its visibility. Another market analysis should be done to see if more advertisement would rise the sales.

*    If we have two similar products that one is expected to be sold more than the other and both are related, such as two types of fruits or bakery, a good marketing idea would be to sell them together for a discounted price. That way we would be encouraging the users to buy them together, getting some economical benefit from both of them and not from just the popular one that they were already going to buy.

*    Following the same idea as the previous point, if the client is not enthusiastic about making a pack with both products, another option would be putting them close, maybe one next to each other if they have a strong relationship.